<h1>human organ classification </h1>
<h2> C-NN model<h3> [transfer learn on MobileNetV2 model]</h3></h2>

call to packages & libraries

In [1]:

from tensorflow import keras
from datetime import datetime

from src.sup.evaluation import *
from src.sup.support import *
from src.sup.test_set_eval import *

from tensorflow.keras.optimizers import RMSprop,SGD,Adam
from tensorflow.keras.layers import Dense,BatchNormalization,Dropout
from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import Model

In [2]:
model_name = "mobileNet_v2_transfer_learn_1.1"

# Load the TensorBoard notebook extension.
%load_ext tensorboard

# call inline plt.

In [ ]:
# Clear any logs from previous runs
!rm -rf ./logs/

callout dataset

In [3]:
classes = ['Heart','Brain','Eye','Kidney','Skull','Other']
root_dir = '../../datasets/'
train_dir = os.path.join(root_dir,'train/')
validation_dir = os.path.join(root_dir,'validation/')
tr_heart_dir,tr_brain_dir,tr_eye_dir,tr_kidney_dir,tr_skull_dir,tr_other_dir = path_update(train_dir,classes)
vl_heart_dir,vl_brain_dir,vl_eye_dir,vl_kidney_dir,vl_skull_dir,vl_other_dir = path_update(validation_dir,classes)

take a glance at training dataset

In [ ]:
plot_sample_of_img(4,4,os.listdir(tr_heart_dir)+os.listdir(tr_eye_dir))

ImageGenator - autolabelling, and categorizing.

In [4]:
train_gen_tmp = ImageDataGenerator(rescale=1./255,
      width_shift_range=0.2,
      height_shift_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

validation_gen_tmp = ImageDataGenerator(rescale=1/225.)

train_gen = train_gen_tmp.flow_from_directory(train_dir,
                                              target_size=(224,224),
                                              color_mode="grayscale",
                                              class_mode='categorical',
                                              batch_size= 20,
                                              shuffle=True,
                                              seed=42)

validation_gen = validation_gen_tmp.flow_from_directory(validation_dir,
                                              target_size=(224,224),
                                              color_mode="grayscale",
                                              class_mode='categorical',
                                              batch_size= 20,
                                              shuffle=True,
                                              seed=42)

STEP_SIZE_TRAIN=train_gen.n//train_gen.batch_size
STEP_SIZE_VALID=validation_gen.n//validation_gen.batch_size

clToInt_dict = train_gen.class_indices
clToInt_dict = dict((k,v) for v,k in clToInt_dict.items())

Found 2803 images belonging to 6 classes.
Found 247 images belonging to 6 classes.


define the model

In [5]:
weight_path = '../../h5_files/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224.h5'

pre_model = MobileNetV2(input_shape=(224,224,1),
                    weights = None,
                    include_top = False)

pre_model.load_weights(weight_path)
#model.summary()

for layer in pre_model.layers:
    layer.trainable = False

conn_layer = pre_model.get_layer('block_12_add')
conn_output = conn_layer.output

x = Conv2D(256,(3,3),activation='relu')(conn_output)
x = MaxPool2D(2,2)(x)
x = Flatten()(x)
#x = Dense(256,activation='relu')(x)
#x = Dropout(0.2)(x)
x = Dense(128,activation='relu')(x)
#x = Dropout(0.2)(x)
#x = BatchNormalization()(x)
x = Dense(6,activation='softmax')(x)

model = Model(pre_model.input,x)

#model.summary()

ValueError: You are trying to load a weight file containing 105 layers into a model with 104 layers.

compile the model

In [ ]:
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

save the log

In [ ]:
# Define the Keras TensorBoard callback.
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")+'/'

if not os.path.exists(logdir):
    os.mkdir(logdir)

#print(datetime.now().strftime("%Y%m%d%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1, profile_batch = 100000000)

fit & train the model.

In [ ]:
history = model.fit(train_gen,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_gen,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10,
                    verbose=1,
                    callbacks= [tensorboard_callback])

visualize layer process in cnn

In [ ]:
#visualize_model(model,img_path)

instant evaluation

In [ ]:
#call to the tensorboard
#%tensorboard --logdir ./logs/fit/
#look at training model performance
acc_n_loss(history)

#model.evaluate_generator(validation_gen,
#                         steps=STEP_SIZE_VALID)

evaluate the model on test set.


In [ ]:
model_path = '../../h5_files/models/20200816005012mobileNet_v2_transfer_learn.h5'
model_weight_path = '../../h5_files/weights/20200816005012mobileNet_v2_transfer_learn.h5'

model = load_model(model_path)
model.load_weights(model_weight_path)
y_pred,y_test = test_eval(model,classes)
plot_confusion_metrix(y_test,y_pred,classes)
ROC_classes(6,y_test,y_pred,classes)



 save the model in .h5 file

In [ ]:
model_path,model_weight_path = save(model,datetime.now().strftime("%Y%m%d%H%M%S")+model_name+'.h5')

make prediction on random images

In [ ]:
model_path = '../../h5_files/models/20200816005012mobileNet_v2_transfer_learn.h5'
model_weight_path = '../../h5_files/weights/20200816005012mobileNet_v2_transfer_learn.h5'
img_path = '../../datasets/validation/../../datasets/validation/Other/11.jpg'
rnd_predict(model_path,model_weight_path,img_path,clToInt_dict)